In [1]:
import ee

In [2]:
ee.Authenticate(force=True)
ee.Initialize()


Successfully saved authorization token.


In [4]:
hybas_lev02  = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroBASINS_lev02/hybas_lev02_v1c_merged_lake_count_added_gt100")
hybas_lev02_lake_count = hybas_lev02.aggregate_array('Lake_count').getInfo()
print(hybas_lev02_lake_count)
hybas_lev02_size = hybas_lev02.size().getInfo()
print(hybas_lev02_size)
hybas_lev02_list = hybas_lev02.toList(hybas_lev02_size)

[617, 64107, 8270, 950, 2671, 3024, 4413, 5453, 59304, 6211, 151400, 15715, 2564, 12745, 14389, 15827, 11029, 2079, 204706, 4653, 1884, 9117, 140061, 3745, 28925, 2569, 3153, 15004, 11615, 43152, 264579, 13525, 19239, 502, 54850, 1481, 26970, 9581, 7444, 52105, 1850, 53019, 7748, 20060, 6668, 1517, 2786, 496, 1591, 4935, 1416, 12733, 2006, 877, 1031, 517, 4742, 588, 1051, 1503]
60


In [4]:
hylak_pnt = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES_points_v10_shp")
hylak_buffered = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES_polys_v10_buffered")
for i in range(hybas_lev02_size):
    current_bas = ee.Feature(hybas_lev02_list.get(i))
    current_bas_id = current_bas.get('HYBAS_ID').getInfo()
    print('Processing ' + str(i) + 'th basin: ' + str(current_bas_id))
    hylak_contained_by_current_bas = hylak_pnt.filterBounds(current_bas.geometry(1))
    current_contained_hylak_id_list = hylak_contained_by_current_bas.aggregate_array('Hylak_id').getInfo()
    print("Filtering hylak_buffered using basin:" + str(current_bas_id) + "...")
    current_hylak_buffered_bound = hylak_buffered.filter(ee.Filter.inList('Hylak_id', current_contained_hylak_id_list))
    task = ee.batch.Export.table.toDrive(
        collection=current_hylak_buffered_bound,
        description='hylak_buffered_' + str(current_bas_id),
        fileFormat='SHP',
        folder='hylak_buffered'
    )
    print('Exporting hylak_buffered_' + str(current_bas_id))
    task.start()

Processing 0th basin: 1020000010
Filtering hylak_buffered using basin:1020000010...
Exporting hylak_buffered_1020000010
Processing 1th basin: 8020000010
Filtering hylak_buffered using basin:8020000010...
Exporting hylak_buffered_8020000010
Processing 2th basin: 4020000010
Filtering hylak_buffered using basin:4020000010...
Exporting hylak_buffered_4020000010
Processing 3th basin: 5020000010
Filtering hylak_buffered using basin:5020000010...
Exporting hylak_buffered_5020000010
Processing 4th basin: 2020000010
Filtering hylak_buffered using basin:2020000010...
Exporting hylak_buffered_2020000010
Processing 5th basin: 9020000010
Filtering hylak_buffered using basin:9020000010...
Exporting hylak_buffered_9020000010
Processing 6th basin: 7020000010
Filtering hylak_buffered using basin:7020000010...
Exporting hylak_buffered_7020000010
Processing 7th basin: 6020000010
Filtering hylak_buffered using basin:6020000010...
Exporting hylak_buffered_6020000010
Processing 8th basin: 3020000010
Filteri

In [11]:
def to_polygon(feature):
    geometry = feature.geometry()
    geom_type = geometry.type()
    
    # Convert Points to Polygons by buffering with a small radius
    point_condition = geom_type.equals('Point')
    multipoint_condition = geom_type.equals('MultiPoint')
    line_condition = geom_type.equals('LineString')
    multiline_condition = geom_type.equals('MultiLineString')
    
    buffered = ee.Algorithms.If(point_condition, geometry.buffer(1), geometry)
    buffered = ee.Algorithms.If(multipoint_condition, geometry.buffer(1), buffered)
    buffered = ee.Algorithms.If(line_condition, geometry.buffer(1), buffered)
    buffered = ee.Algorithms.If(multiline_condition, geometry.buffer(1), buffered)
    
    return feature.setGeometry(ee.Geometry(buffered))

def buffer_1(feature):
    return feature.buffer(1)

hylak_pnt = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES_points_v10_shp")
hylak_poly = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES_polys_v10")
for i in range(hybas_lev02_size):
    if i < 24:
        continue
    current_bas = ee.Feature(hybas_lev02_list.get(i))
    current_bas_id = current_bas.get('HYBAS_ID').getInfo()
    print('Processing ' + str(i) + 'th basin: ' + str(current_bas_id))
    hylak_contained_by_current_bas = hylak_pnt.filterBounds(current_bas.geometry(1))
    current_contained_hylak_id_list = hylak_contained_by_current_bas.aggregate_array('Hylak_id').getInfo()
    print("Filtering hylak_poly using basin:" + str(current_bas_id) + "...")
    
    current_hylak_poly = hylak_poly.filter(ee.Filter.inList('Hylak_id', current_contained_hylak_id_list))
    current_hylak_poly = current_hylak_poly.map(buffer_1)
    
    task = ee.batch.Export.table.toDrive(
        collection=current_hylak_poly,
        description='hylak_unbuffered_' + str(current_bas_id),
        fileFormat='SHP',
        folder='hylak_unbuffered'
    )
    print('Exporting hylak_unbuffered_' + str(current_bas_id))
    task.start()

Processing 24th basin: 3020005240
Filtering hylak_poly using basin:3020005240...
Exporting hylak_unbuffered_3020005240
Processing 25th basin: 1020021940
Filtering hylak_poly using basin:1020021940...
Exporting hylak_unbuffered_1020021940
Processing 26th basin: 8020020760
Filtering hylak_poly using basin:8020020760...
Exporting hylak_unbuffered_8020020760
Processing 27th basin: 4020024190
Filtering hylak_poly using basin:4020024190...
Exporting hylak_unbuffered_4020024190
Processing 28th basin: 5020049720
Filtering hylak_poly using basin:5020049720...
Exporting hylak_unbuffered_5020049720
Processing 29th basin: 2020024230
Filtering hylak_poly using basin:2020024230...
Exporting hylak_unbuffered_2020024230
Processing 30th basin: 7020024600
Filtering hylak_poly using basin:7020024600...
Exporting hylak_unbuffered_7020024600
Processing 31th basin: 6020014330
Filtering hylak_poly using basin:6020014330...
Exporting hylak_unbuffered_6020014330
Processing 32th basin: 3020008670
Filtering hyla

In [ ]:
hylak_pnt = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES_points_v10_shp")
hylak_buffered_bounds = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES_buffered_bounds")
for i in range(hybas_lev02_size):
    current_bas = ee.Feature(hybas_lev02_list.get(i))
    current_bas_id = current_bas.get('HYBAS_ID').getInfo()
    print('Processing ' + str(i) + 'th basin: ' + str(current_bas_id))
    hylak_contained_by_current_bas = hylak_pnt.filterBounds(current_bas.geometry(1))
    current_contained_hylak_id_list = hylak_contained_by_current_bas.aggregate_array('Hylak_id').getInfo()
    print("Filtering hylak_buffered_bounds using basin:" + str(current_bas_id) + "...")
    current_hylak_buffered_bound = hylak_buffered_bounds.filter(ee.Filter.inList('Hylak_id', current_contained_hylak_id_list))
    task = ee.batch.Export.table.toAsset(
        collection=current_hylak_buffered_bound,
        description='hylak_buffered_bound_' + str(current_bas_id),
        assetId='projects/xuzhihongqu/assets/HydroLAKES/hylak_buffered_bounds_' + str(current_bas_id)
    )
    print('Exporting hylak_buffered_bound_' + str(current_bas_id))
    task.start()

In [ ]:
hylak_pnt = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES_points_v10_shp")
hylak_buffered_bounds = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES_buffered_bounds")
for i in range(hybas_lev02_size):
    current_bas = ee.Feature(hybas_lev02_list.get(i))
    current_bas_id = current_bas.get('HYBAS_ID').getInfo()
    print(type(current_bas_id))
    if current_bas_id not in [4020050290, 4020050470]:
        continue
    print('Processing ' + str(i) + 'th basin: ' + str(current_bas_id))
    hylak_contained_by_current_bas = hylak_pnt.filterBounds(current_bas.geometry(1))
    current_contained_hylak_id_list = hylak_contained_by_current_bas.aggregate_array('Hylak_id').getInfo()
    print("Filtering hylak_buffered_bounds_dissolved using basin:" + str(current_bas_id) + "...")
    current_hylak_buffered_bound = hylak_buffered_bounds.filter(ee.Filter.inList('Hylak_id', current_contained_hylak_id_list)).geometry(1).dissolve(1)
    task = ee.batch.Export.table.toAsset(
        collection=ee.FeatureCollection([ee.Feature(current_hylak_buffered_bound, None)]),
        description='hylak_buffered_bounds_dissolved_' + str(current_bas_id),
        assetId='projects/xuzhihongqu/assets/HydroLAKES_per_basin_dissolved/hylak_buffered_bounds_dissolved_' + str(current_bas_id)
    )
    print('Exporting hylak_buffered_bounds_dissolved_' + str(current_bas_id))
    task.start()

In [3]:
sub_numbers = 4
for i in range(sub_numbers):
    current_bas_id = 7020021430
    print('Processing ' + str(i+1) + 'th sub for basin: ' + str(current_bas_id))
    current_hylak_buffered_bounds = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES/hylak_buffered_bounds_" + str(current_bas_id) + "_sub" + str(i+1))
    current_hylak_buffered_bound = current_hylak_buffered_bounds.geometry(1).dissolve(1)
    task = ee.batch.Export.table.toAsset(
        collection=ee.FeatureCollection([ee.Feature(current_hylak_buffered_bound, None)]),
        description='hylak_buffered_bounds_dissolved_' + str(current_bas_id) + "_sub" + str(i+1),
        assetId='projects/xuzhihongqu/assets/HydroLAKES_per_basin_dissolved/hylak_buffered_bounds_dissolved_' + str(current_bas_id) + "_sub" + str(i+1)
    )
    print('Exporting hylak_buffered_bounds_dissolved_' + str(current_bas_id))
    task.start()

Processing 1th sub for basin: 7020021430
Exporting hylak_buffered_bounds_dissolved_7020021430
Processing 2th sub for basin: 7020021430
Exporting hylak_buffered_bounds_dissolved_7020021430
Processing 3th sub for basin: 7020021430
Exporting hylak_buffered_bounds_dissolved_7020021430
Processing 4th sub for basin: 7020021430
Exporting hylak_buffered_bounds_dissolved_7020021430


In [4]:
sub_numbers = 4
for i in range(sub_numbers):
    current_bas_id = 7020021430
    print('Processing ' + str(i+1) + 'th sub for basin: ' + str(current_bas_id))
    current_hylak_buffered_bounds = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES/hylak_buffered_bounds_" + str(current_bas_id) + "_sub" + str(i+1))
    current_hylak_buffered_bound = current_hylak_buffered_bounds.geometry(1).dissolve(1).simplify(1000)
    task = ee.batch.Export.table.toAsset(
        collection=ee.FeatureCollection([ee.Feature(current_hylak_buffered_bound, None)]),
        description='hylak_buffered_bounds_dissolved_simplified_1000m_' + str(current_bas_id) + "_sub" + str(i+1),
        assetId='projects/xuzhihongqu/assets/HydroLAKES_per_basin_dissolved/hylak_buffered_bounds_dissolved_simplified_1000m_' + str(current_bas_id) + "_sub" + str(i+1)
    )
    print('Exporting hylak_buffered_bounds_dissolved_' + str(current_bas_id))
    task.start()

Processing 1th sub for basin: 7020021430
Exporting hylak_buffered_bounds_dissolved_7020021430
Processing 2th sub for basin: 7020021430
Exporting hylak_buffered_bounds_dissolved_7020021430
Processing 3th sub for basin: 7020021430
Exporting hylak_buffered_bounds_dissolved_7020021430
Processing 4th sub for basin: 7020021430
Exporting hylak_buffered_bounds_dissolved_7020021430


In [ ]:
hybas_gt100_id_list = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroBASINS_lev02/hybas_lev02_v1c_merged_lake_count_added_gt100").aggregate_array("HYBAS_ID").getInfo()
for current_hybas_id in hybas_gt100_id_list:
    current_fc = ee.FeatureCollection('projects/xuzhihongqu/assets/HydroLAKES/hylak_buffered_bounds_' + str(current_hybas_id))
    task = ee.batch.Export.table.toDrive(
        collection=current_fc,
        description='hylak_buffered_bounds_' + str(current_hybas_id),
        fileFormat='SHP',
        folder='hylak_buffered_bounds_SHP'
    )
    print('Exporting hylak_buffered_bounds_' + str(current_hybas_id))
    task.start()

In [ ]:
hybas_gt100_id_list = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroBASINS_lev02/hybas_lev02_v1c_merged_lake_count_added_gt100").aggregate_array("HYBAS_ID").getInfo()
for current_hybas_id in hybas_gt100_id_list:
    current_fc = ee.FeatureCollection('projects/xuzhihongqu/assets/HydroLAKES_per_basin_dissolved/hylak_buffered_bounds_dissolved_' + str(current_hybas_id))
    task = ee.batch.Export.table.toAsset(
        collection=ee.FeatureCollection(ee.Feature(current_fc.geometry(1).simplify(ee.ErrorMargin(1000)))),
        description=f'{current_hybas_id}_hylak_dissolved_simplified_1000m',
        assetId=f"projects/xuzhihongqu/assets/HydroLAKES_per_basin_dissolved_simplified/hylak_buffered_bounds_dissolved_simplified_1000m_{current_hybas_id}",
    )
    print('Exporting hylak_buffered_bounds_dissolved_simplified_1000m_' + str(current_hybas_id))
    task.start()

In [ ]:
hybas_lev02  = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroBASINS_lev02/hybas_lev02_v1c_merged_lake_count_added_gt100")
hybas_lev02_lake_count = hybas_lev02.aggregate_array('Lake_count').getInfo()
print(hybas_lev02_lake_count)
hybas_lev02_size = hybas_lev02.size().getInfo()
print(hybas_lev02_size)
hybas_lev02_list = hybas_lev02.toList(hybas_lev02_size)
for i in range(hybas_lev02_size):
    current_bas = ee.Feature(hybas_lev02_list.get(i))
    current_bas_id = current_bas.get('HYBAS_ID').getInfo()
    task = ee.batch.Export.table.toDrive(
        collection=ee.FeatureCollection(current_bas),
        description='hybas_lev02_' + str(current_bas_id),
        fileFormat='SHP',
        folder='hybas_lev02_shp'
    )
    print('Exporting hybas_lev02_' + str(current_bas_id))
    task.start()

In [13]:
# Create for GLAKES-updated HydroLAKES
hybas_id_list = [
    1020000010, 1020011530, 1020018110, 1020021940, 1020027430, 1020034170, 1020035180, 1020040190,
    2020000010, 2020003440, 2020018240, 2020024230, 2020033490, 2020041390, 2020057170, 2020065840, 2020071190,
    3020000010, 3020003790, 3020005240, 3020008670, 3020009320, 3020024310,
    4020000010, 4020006940, 4020015090, 4020024190, 4020034510, 4020050210, 4020050220, 4020050290, 4020050470,
    5020000010, 5020015660, 5020037270, 5020049720, 5020082270, 
    6020000010, 6020006540, 6020008320, 6020014330, 6020017370, 6020021870, 6020029280,
    7020000010, 7020014250, 7020021430, 7020024600, 7020038340, 7020046750, 7020047840, 7020065090,
    8020000010, 8020008900, 8020010700, 8020020760, 8020022890, 8020032840, 8020044560,
    9020000010
]
hylak_per_basin_asset_id_pattern = 'projects/xuzhihongqu/assets/HydroLAKES_GLAKES_updated_unbuffered/hylak_unbuffered_updated_{hybas_id}'


for hybas_id in hybas_id_list:
    current_hylak_asset_id = hylak_per_basin_asset_id_pattern.format(hybas_id=hybas_id)
    current_hylak_fc = ee.FeatureCollection(current_hylak_asset_id)
    lake_area_min = ee.Number(0.1)
    lake_area_max = ee.Number(1000)
    slope_100_min = ee.Number(current_hylak_fc.aggregate_min('Slope_100'))
    slope_100_max = ee.Number(current_hylak_fc.aggregate_max('Slope_100'))
    buffer_min = ee.Number(120)
    buffer_max = ee.Number(1250)
    def create_buffer(x):
        slope_100 = ee.Number(x.get('Slope_100'))
        lake_area = ee.Number(x.get('Lake_area'))
        buffer_adjust_ratio = lake_area.subtract(lake_area_min).divide(lake_area_max.subtract(lake_area_min))
        slope_inflation_factor = ee.Number(ee.Algorithms.If(slope_100.eq(-1), 0.5, slope_100_max.subtract(slope_100).divide(slope_100_max.subtract(slope_100_min))))
        slope_inflation_factor = slope_inflation_factor.add(1)
        buffer_size = buffer_min.add(buffer_adjust_ratio.multiply(buffer_max.subtract(buffer_min))).multiply(slope_inflation_factor)
        return x.buffer(buffer_size, 1)
    current_hylak_fc_buffered = current_hylak_fc.map(create_buffer)
    task = ee.batch.Export.table.toDrive(
        collection=current_hylak_fc_buffered,
        description='hylak_buffered_GLAKES_updated_' + str(hybas_id),
        folder='hylak_GLAKES_updated_buffered',
        fileFormat='SHP'
    )
    print(f'Exporting hylak_buffered_GLAKES_updated_{hybas_id}')
    task.start()
    

Exporting hylak_buffered_GLAKES_updated_1020000010
Exporting hylak_buffered_GLAKES_updated_1020011530
Exporting hylak_buffered_GLAKES_updated_1020018110
Exporting hylak_buffered_GLAKES_updated_1020021940
Exporting hylak_buffered_GLAKES_updated_1020027430
Exporting hylak_buffered_GLAKES_updated_1020034170
Exporting hylak_buffered_GLAKES_updated_1020035180
Exporting hylak_buffered_GLAKES_updated_1020040190
Exporting hylak_buffered_GLAKES_updated_2020000010
Exporting hylak_buffered_GLAKES_updated_2020003440
Exporting hylak_buffered_GLAKES_updated_2020018240
Exporting hylak_buffered_GLAKES_updated_2020024230
Exporting hylak_buffered_GLAKES_updated_2020033490
Exporting hylak_buffered_GLAKES_updated_2020041390
Exporting hylak_buffered_GLAKES_updated_2020057170
Exporting hylak_buffered_GLAKES_updated_2020065840
Exporting hylak_buffered_GLAKES_updated_2020071190
Exporting hylak_buffered_GLAKES_updated_3020000010
Exporting hylak_buffered_GLAKES_updated_3020003790
Exporting hylak_buffered_GLAKES